In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/FillZero_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # Chỉ sử dụng một lớp Bidirectional LSTM
        x = layers.Bidirectional(layers.LSTM(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=False  # Thay đổi thành False vì đây là lớp LSTM duy nhất
        ))(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'bilstm_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 02m 28s]
val_accuracy: 0.7974374890327454

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 25m 19s
Best Parameters for Week 1:
units_1: 192
dropout_1: 0.30000000000000004
learning_rate: 0.0008973681703772068

Best Parameters for Week 2:
units_1: 160
dropout_1: 0.30000000000000004
learning_rate: 0.000768513725396424

Best Parameters for Week 3:
units_1: 160
dropout_1: 0.1
learning_rate: 0.00031802754751204213

Best Parameters for Week 4:
units_1: 224
dropout_1: 0.4
learning_rate: 0.0012918827423762096


## Danh sách tham số tốt nhất của từng tuần

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# # Xây dựng mô hình BiLSTM
# def build_Bilstm_model(params, input_shape):
#     inputs = tf.keras.Input(shape=input_shape)  # Định nghĩa đầu vào
    
#     # Bidirectional LSTM layer 1
#     x = layers.Bidirectional(layers.LSTM(
#         units=params.get('units_1'),
#         return_sequences=True
#     ))(inputs)
#     x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
#     # Bidirectional LSTM layer 2
#     x = layers.Bidirectional(layers.LSTM(
#         units=params.get('units_2', 32),
#         return_sequences=False
#     ))(x)
#     x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
#     # Lớp đầu ra
#     outputs = layers.Dense(5, activation='softmax')(x)
    
#     # Khởi tạo mô hình
#     model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
#     # Compile với Focal Loss
#     model.compile(optimizer=tf.keras.optimizers.Adam(
#                       learning_rate=params['learning_rate']),
#                   loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
#                   metrics=['accuracy'])
    
#     return model

def build_Bilstm_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)  # Định nghĩa đầu vào
    
    # Chỉ sử dụng một lớp Bidirectional LSTM
    x = layers.Bidirectional(layers.LSTM(
        units=params.get('units_1', 64),
        return_sequences=False  # Đặt thành False vì đây là lớp LSTM cuối cùng
    ))(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
        
        # Xây dựng mô hình với tham số tốt nhất
        input_shape = (X_train.shape[1], X_train.shape[2])
        model = build_Bilstm_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 192, 'dropout_1': 0.30000000000000004, 'learning_rate': 0.0008973681703772068}
Fold 1: Using file /kaggle/input/FillZero_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5971 - loss: 0.1518 - val_accuracy: 0.7061 - val_loss: 0.1100
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6738 - loss: 0.1172 - val_accuracy: 0.6927 - val_loss: 0.1082
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6928 - loss: 0.1139 - val_accuracy: 0.7179 - val_loss: 0.1048
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6977 - loss: 0.1122 - val_accuracy: 0.7137 - val_loss: 0.1038
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6977 - loss: 0.1091 - val_accuracy: 0.7209 - val_loss: 0.1028
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7069 - loss: 0.1074 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5907 - loss: 0.1483 - val_accuracy: 0.7045 - val_loss: 0.1108
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6765 - loss: 0.1173 - val_accuracy: 0.6893 - val_loss: 0.1113
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6932 - loss: 0.1137 - val_accuracy: 0.7148 - val_loss: 0.1085
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6980 - loss: 0.1106 - val_accuracy: 0.7316 - val_loss: 0.1067
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7024 - loss: 0.1090 - val_accuracy: 0.7240 - val_loss: 0.1085
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6944 - loss: 0.1118 - val_accuracy: 0.7194 - val_loss: 0.1074
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7110 - loss: 0.1069 - val_accuracy: 0.7411 - val_loss: 0.1038
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7114 - loss: 0.1075 - val_accuracy: 0.7160 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 4: Using file /kaggle/input/FillZero_minmax_baseline/clean_week1/train/5-folds/data_part_4.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6056 - loss: 0.1447 - val_accuracy: 0.6968 - val_loss: 0.1086
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6803 - loss: 0.1182 - val_accuracy: 0.7201 - val_loss: 0.1074
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6877 - loss: 0.1150 - val_accuracy: 0.7201 - val_loss: 0.1054
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6946 - loss: 0.1103 - val_accuracy: 0.7113 - val_loss: 0.1057
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6925 - loss: 0.1122 - val_accuracy: 0.7178 - val_loss: 0.1054
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6957 - loss: 0.1102 - val_accuracy: 0.7250 - val_loss: 0.1049
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7041 - loss: 0.1080 - val_accuracy: 0.7285 - val_loss:

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 3: Using file /kaggle/input/FillZero_minmax_baseline/clean_week4/train/5-folds/data_part_3.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6267 - loss: 0.1354 - val_accuracy: 0.6873 - val_loss: 0.1145
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6842 - loss: 0.1171 - val_accuracy: 0.6827 - val_loss: 0.1146
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6988 - loss: 0.1121 - val_accuracy: 0.6953 - val_loss: 0.1159
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6991 - loss: 0.1104 - val_accuracy: 0.6907 - val_loss: 0.1112
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6991 - loss: 0.1132 - val_accuracy: 0.7170 - val_loss: 0.1049
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7027 - loss: 0.1107 - val_accuracy: 0.7128 - val_loss: 0.1067
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7146 - loss: 0.1072 - val_accuracy: 0.7212 - val_loss:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 5: Using file /kaggle/input/FillZero_minmax_baseline/clean_week4/train/5-folds/data_part_5.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6176 - loss: 0.1423 - val_accuracy: 0.6735 - val_loss: 0.1111
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6749 - loss: 0.1219 - val_accuracy: 0.6861 - val_loss: 0.1134
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6867 - loss: 0.1146 - val_accuracy: 0.6842 - val_loss: 0.1094
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6874 - loss: 0.1131 - val_accuracy: 0.7113 - val_loss: 0.1050
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6936 - loss: 0.1128 - val_accuracy: 0.7174 - val_loss: 0.1035
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6996 - loss: 0.1140 - val_accuracy: 0.7265 - val_loss: 0.1033
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7002 - loss: 0.1123 - val_accuracy: 0.7288 - val_loss:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Kết quả cross validation trên 5-folds

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.7661684393882752
Average Train Time: 86.0488 seconds
Average Test Time: 0.4899 seconds
Average AUC Macro: 0.8583054238209586
Average AUC Weighted: 0.8838356710288648

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.647584        0.616000          0.630453     0.855262
1      1           0.200000        0.002299          0.004545     0.825179
2      2           0.734320        0.458566          0.552168     0.856203
3      3           0.655920        0.346108          0.433217     0.873669
4      4           0.795548        0.917172          0.851513     0.881095

Average Confusion Matrix:
       0    1     2     3       4
0  369.6  0.0   9.4   7.4   213.6
1   44.2  0.2   0.6   4.4    38.2
2   36.4  0.2  75.4   4.0    48.4
3   27.0  0.2   2.2  57.8    79.8
4   95.4  0.2  18.2  19.0  1470.6

=== Results for week2 ===
Average Accurancy: 0.

## Kiểm tra trên tập test

In [11]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])
    model = build_Bilstm_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [12]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4809 - loss: 0.1979 - val_accuracy: 0.4744 - val_loss: 0.2911
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5565 - loss: 0.1737 - val_accuracy: 0.4744 - val_loss: 0.2660
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5767 - loss: 0.1674 - val_accuracy: 0.4368 - val_loss: 0.2825
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5784 - loss: 0.1661 - val_accuracy: 0.4410 - val_loss: 0.2987
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5817 - loss: 0.1620 - val_accuracy: 0.4951 - val_loss: 0.2619
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5863 - loss: 0.1611 - val_accuracy: 0.6463 - val_loss: 0.2149
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5902 - loss: 0.1574 - val_accuracy: 0.5600 - val_loss: 0.2437
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [13]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4811 - loss: 0.1962 - val_accuracy: 0.5171 - val_loss: 0.2296
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5625 - loss: 0.1706 - val_accuracy: 0.3944 - val_loss: 0.3076
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5813 - loss: 0.1639 - val_accuracy: 0.6109 - val_loss: 0.2257
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5911 - loss: 0.1589 - val_accuracy: 0.4819 - val_loss: 0.2402
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6006 - loss: 0.1555 - val_accuracy: 0.6610 - val_loss: 0.1873
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5965 - loss: 0.1550 - val_accuracy: 0.6426 - val_loss: 0.2064
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5985 - loss: 0.1552 - val_accuracy: 0.5974 - val_loss: 0.2152
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [14]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.4923 - loss: 0.1937 - val_accuracy: 0.4368 - val_loss: 0.2800
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5909 - loss: 0.1636 - val_accuracy: 0.5166 - val_loss: 0.2510
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6032 - loss: 0.1537 - val_accuracy: 0.6358 - val_loss: 0.2233
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6181 - loss: 0.1484 - val_accuracy: 0.5770 - val_loss: 0.2447
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6271 - loss: 0.1437 - val_accuracy: 0.6401 - val_loss: 0.2185
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6339 - loss: 0.1401 - val_accuracy: 0.5755 - val_loss: 0.2481
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6291 - loss: 0.1414 - val_accuracy: 0.5977 - val_loss: 0.2221
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accurac

In [15]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4660 - loss: 0.2018 - val_accuracy: 0.4355 - val_loss: 0.3074
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5296 - loss: 0.1794 - val_accuracy: 0.5276 - val_loss: 0.2369
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5531 - loss: 0.1717 - val_accuracy: 0.4395 - val_loss: 0.2776
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5573 - loss: 0.1696 - val_accuracy: 0.5081 - val_loss: 0.2460
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5695 - loss: 0.1659 - val_accuracy: 0.4719 - val_loss: 0.2691
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5738 - loss: 0.1620 - val_accuracy: 0.5570 - val_loss: 0.2129
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5766 - loss: 0.1621 - val_accuracy: 0.6403 - val_loss: 0.2152
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [16]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 273.58 seconds
  Test Time: 0.44 seconds
  Accurancy: 0.6871951219512196
  Precision: [0.6173913  0.18181818 0.50877193 0.48387097 0.90322581]
  Recall: [0.568      0.74074074 0.5631068  0.57142857 0.75373878]
  F1-Score: [0.59166667 0.2919708  0.53456221 0.52401747 0.82173913]
  Macro Precision: 0.5390156380368236
  Macro Recall: 0.6394029783869739
  Macro F1-Score: 0.5527912558503265
  Confusion Matrix:
[[213  87  21   5  49]
 [  8  40   0   1   5]
 [ 19  15  58   2   9]
 [  5  14   8  60  18]
 [100  64  27  56 756]]

=== AUC-ROC ===
AUC Macro: 0.8674
AUC Weighted: 0.8687
AUC per Label: [0.83489223 0.8810191  0.84949245 0.89119591 0.88024701]
Week 2:
  Train Time: 278.02 seconds
  Test Time: 0.43 seconds
  Accurancy: 0.698170731707317
  Precision: [0.76315789 0.15272727 0.50420168 0.51401869 0.89919817]
  Recall: [0.54133333 0.77777778 0.58252427 0.52380952 0.78265204]
  F1-Score: [0.63338534 0.25531915 0.54054054 0.5188679